In [1]:
!pip install -q pyomo
!apt-get install -y -qq coinor-cbc

     |████████████████████████████████| 9.1 MB 4.2 MB/s 
     |████████████████████████████████| 49 kB 5.7 MB/s 
Selecting previously unselected package coinor-libcoinutils3v5.
(Reading database ... 148492 files and directories currently installed.)
Preparing to unpack .../0-coinor-libcoinutils3v5_2.10.14+repack1-1_amd64.deb ...
Unpacking coinor-libcoinutils3v5 (2.10.14+repack1-1) ...
Selecting previously unselected package coinor-libosi1v5.
Preparing to unpack .../1-coinor-libosi1v5_0.107.9+repack1-1_amd64.deb ...
Unpacking coinor-libosi1v5 (0.107.9+repack1-1) ...
Selecting previously unselected package coinor-libclp1.
Preparing to unpack .../2-coinor-libclp1_1.16.11+repack1-1_amd64.deb ...
Unpacking coinor-libclp1 (1.16.11+repack1-1) ...
Selecting previously unselected package coinor-libcgl1.
Preparing to unpack .../3-coinor-libcgl1_0.59.10+repack1-1_amd64.deb ...
Unpacking coinor-libcgl1 (0.59.10+repack1-1) ...
Selecting previously unselected package coinor-libcbc3.
Preparing to unp

### 1)

Given constraint is
$ |(x_1 + x_2) - (x_3+x_4+x_5) | \leq 18 \\
\implies -18 \leq (x_1 + x_2) - (x_3+x_4+x_5) \leq 18\\
\implies (x_1 + x_2) - (x_3+x_4+x_5) \leq 18,\  and\qquad (x_1 + x_2) - (x_3+x_4+x_5) \geq 18$

We can incorporate these 2 linear constraints in our LP

The new LP can be formulated as:

\begin{align}
\max \ 35 x_1 + 57 x_2 + 48 x_3 + 20 x_4 + 15 x_5, \\
{\rm{s.t.}} \; x_1 + x_2 + x_3 & \leq 75, \\
x_3 + x_4 + x_5 & \leq 100, \\
x_1 + x_2 & \geq 10, \\ 
x_1 + x_2 - x_3 - x_4 - x_5 & \leq 18, \\
x_1 + x_2 - x_3 - x_4 - x_5 & \geq -18, \\
30000 x_1 + 95000 x_2 + 10000 x_3 + 5000 x_4 + 4000 x_5 & \leq 1500000, \\
100000 x_1 + 500000 x_2 + 200000 x_3 + 45000 x_4 + 25000 x_5 & \geq 900000, \\
x_i & \geq 0, \quad i \in \{1,2,3,4,5\}. \\
x_i \in \mathbb{Z}
\end{align}

The given formulation is a linear program, since objection function and all constraints are linear.

In [2]:
from pyomo.environ import *
import numpy as np


In [3]:
objcoef = [35,57,48,20,15]
conscoef = np.array([[1,1,1,0,0],[0,0,1,1,1],[-1,-1,0,0,0],[1,1,-1,-1,-1],[-1,-1,1,1,1],[30000,95000,10000,5000,4000],[-100000,-500000,-200000,-45000,-25000]])
consrhs = [75,100,-10,18,18,1500000,-900000]

n = 5
m = 7
cols= np.arange(n)
rows = np.arange(m)

media= ['Website','TV','daily newspaper','weekly magazine','monthly magazine']

In [4]:
m2 = ConcreteModel()
m2.x = Var(cols,domain=NonNegativeIntegers)
m2.cons = ConstraintList()

In [5]:
for i in rows:
    m2.cons.add(summation(conscoef[i],m2.x)<=consrhs[i])
    
m2.obj = Objective(expr=summation(objcoef,m2.x),sense=maximize)    

In [6]:
solver = SolverFactory('cbc')

In [7]:
opt1 = solver.solve(m2)

In [8]:
# display solution
print('Optimal value of objective function = ', m2.obj())

print('\nDecision Variables')
for i in cols:
    print('Number of slots for ',media[i],'are', m2.x[i].value)

Optimal value of objective function =  3378.0

Decision Variables
Number of slots for  Website are 34.0
Number of slots for  TV are 0.0
Number of slots for  daily newspaper are 41.0
Number of slots for  weekly magazine are 11.0
Number of slots for  monthly magazine are 0.0


In [9]:
print('\nConstraints')
m2.cons.display()
print('Constraint 1 and constraint 5 are active constraints ')


Constraints
cons : Size=7
    Key : Lower : Body        : Upper
      1 :  None :        75.0 :      75.0
      2 :  None :        52.0 :     100.0
      3 :  None :       -34.0 :     -10.0
      4 :  None :       -18.0 :      18.0
      5 :  None :        18.0 :      18.0
      6 :  None :   1485000.0 : 1500000.0
      7 :  None : -12095000.0 : -900000.0
Constraint 1 and constraint 5 are active constraints 


### 7)

The new optimisation problem can be formulated as:

\begin{align}
\max \ 35 x_1 + 57 x_2 + 48 x_3 + 20 x_4 + 15 x_5, \\
{\rm{s.t.}} \; x_1 + x_2 + x_3 & \leq 75, \\
x_3 + x_4 + x_5 & \leq 100, \\
x_1 + x_2 & \geq 10, \\ 
|x_4 + x_5 - x_3| & \geq 4, \\
30000 x_1 + 95000 x_2 + 10000 x_3 + 5000 x_4 + 4000 x_5 & \leq 1500000, \\
100000 x_1 + 500000 x_2 + 200000 x_3 + 45000 x_4 + 25000 x_5 & \geq 900000, \\
x_i & \geq 0, \quad i \in \{1,2,3,4,5\}. \\
x_i \in \mathbb{Z}
\end{align}

This is not a linear program since one of the constraints has an absolute value function, which cannot be decomposed into linear functions in the same variables. 

Let $y = |x_4 + x_5 - x_3|$\
hence,
$y \geq x_4 + x_5 - x_3,\\
y \geq - x_4 - x_5 + x_3$

\begin{align}
\max \ 35 x_1 + 57 x_2 + 48 x_3 + 20 x_4 + 15 x_5, \\
{\rm{s.t.}} \; x_1 + x_2 + x_3 & \leq 75, \\
x_3 + x_4 + x_5 & \leq 100, \\
x_1 + x_2 & \geq 10, \\ 
y - x_4 - x_5 + x_3 & \geq 0, \\
y + x_4 + x_5 - x_3 & \geq 0, \\
y & \geq 4, \\
30000 x_1 + 95000 x_2 + 10000 x_3 + 5000 x_4 + 4000 x_5 & \leq 1500000, \\
100000 x_1 + 500000 x_2 + 200000 x_3 + 45000 x_4 + 25000 x_5 & \geq 900000, \\
x_i & \geq 0, \quad i \in \{1,2,3,4,5\}. \\
x_i \in \mathbb{Z}
\end{align}